# Playground space

Espaço para testes, PoC e validação de hipóteses.

In [ ]:
%load_ext blackcellmagic

In [ ]:
## Importação de dependências
import os
import re
import pandas as pd
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
from dataclasses import dataclass, field
from typing import Optional, List, Dict
import uuid
from datetime import datetime, timezone
import json
import math
from fuzzywuzzy import process
from selector import environment_selector
from firestore_query_batch import query_batch
from firestore_model_abrigado_legacy import AbrigadoLegacyEntity, MembroFamiliarLegacyEntity, DocumentoLegacyEntity, ResponsavelLegacyEntity
from firestore_model_abrigo_legacy import AbrigoLegacyEntity
from nome import normalize_nome
from abrigo_dict import get_abrigo_info

## Configuração

### Environment Selector

In [ ]:
environment_selector()

### Configura chaves e parâmetros

In [ ]:
# Firebase Firestore Config
base_path = os.getenv("SOSRS_BASEPATH")
sosrs_environment = os.getenv("SOSRS_ENVIRONMENT")
sosrs_firestore_keyfile = os.getenv("SOSRS_FIRESTORE_KEYFILE")
sosrs_firestore_path = os.path.join(base_path, sosrs_firestore_keyfile)

# Airtable Config
sosrs_airtable_api_key = os.getenv("SOSRS_AIRTABLE_API_KEY")

### Autenticação do notebook

In [ ]:
# Inicialização do Firebase
cred = credentials.Certificate(sosrs_firestore_path)

firebase_admin.initialize_app(cred)

client_firestore = firestore.client()

## Manipulação de documentos

### Adiciona um novo Abrigo

In [ ]:
abrigo_entity = AbrigoLegacyEntity(
    create_in=datetime.now(timezone.utc),
    update_in=datetime.now(timezone.utc),
    tipo="Abrigo",
    city="Porto Alegre",
    nome="Abrigo não informado na importação",
)


try:
    doc_ref = client_firestore.collection("abrigo").document()
    abrigo_entity.set_id(doc_ref.id)
    print(abrigo_entity.id)
    doc_ref.set(abrigo_entity.to_dict())
except Exception as e:
    print(f"Erro ao enviar dados: {e}")

### Soma total de abrigados, incluindo grupo familiar

In [ ]:
# Referencia a coleção
collection_ref = client_firestore.collection("abrigado")
documents = collection_ref.stream()

# Contadores para nomes
total_non_empty_names = 0
# Itera sobre cada documento na coleção
for document in documents:
    data = document.to_dict()
    # Conta o nome do AbrigadoLegacyEntity se não for nulo ou vazio
    if "nome" in data and isinstance(data["nome"], str) and data["nome"].strip():
        total_non_empty_names += 1
    # Verifica se grupoFamiliar existe, não é None, é uma lista, e conta os nomes não nulos/vazios
    if "grupoFamiliar" in data and isinstance(data["grupoFamiliar"], list):
        for membro in data["grupoFamiliar"]:
            if "nome" in membro and isinstance(membro["nome"], str) and membro["nome"].strip():
                total_non_empty_names += 1

print(f"Total de nomes não nulos ou vazios: {total_non_empty_names}")

### Consulta abrigos por nome (like)

In [ ]:
collection_ref = client_firestore.collection('abrigo')

search_text = 'enjoy'
start_at = f'{search_text}'
end_at = f'{search_text}\uf8ff'

query = collection_ref.where('nome', '>=', start_at).where('nome', '<=', end_at)
results = query.stream()

for doc in results:
    doc_dict = doc.to_dict()
    print(f'id: {doc.id}, nome: {doc_dict.get("nome", "Nome não encontrado")}')


### Consulta abrigos por nome (equal)

In [ ]:
collection_ref = client_firestore.collection('abrigado95')

search_text = '31KJeGHcEMeumEcnLFoF'

query = collection_ref.where('id', '==', search_text)
results = query.stream()

for doc in results:
    doc_dict = doc.to_dict()
    print(doc_dict)


### Consulta abrigos por nome (fuzzy)

In [ ]:
search_text = "castelo"

collection_ref = client_firestore.collection("abrigo")
docs = collection_ref.stream()

all_docs = {}
for doc in docs:
    doc_dict = doc.to_dict()
    nome = doc_dict.get("nome")
    if nome:
        # Split words and remove duplicates
        words = set(nome.split())
        for word in words:
            all_docs[word] = { doc.id, nome }

nome_similar, score = process.extractOne(search_text, list(all_docs.keys()))
if score >= 80:
    similar_doc_id = all_docs[nome_similar]
    print(f"nome similar: {nome_similar}, score: {score}, nome: {all_docs[nome_similar]}")

## Ajuste de dados em massa

### Ajusta dados via arquivo

In [ ]:
try:
    datasource_file = "./pipeline/.working/ajuste_nome_sem_abrigo.xlsx"
    collection_name = "abrigado"

    xlsx = pd.ExcelFile(datasource_file)
    data_source = pd.read_excel(xlsx, header=0, names=["abrigo_nome", "abrigado_nome"], usecols=range(2))
    data_source = data_source[data_source["abrigado_nome"].notna() & (data_source["abrigado_nome"] != "")]
    data_source["search_field_name"] = data_source.apply(lambda row: normalize_nome(row["abrigado_nome"]), axis=1)
    registers = data_source.to_dict(orient="records")

    batch = client_firestore.batch()
    count = 0

    for value in registers:
        abrigo_nome, abrigo_id = get_abrigo_info(value["abrigo_nome"])
        
        if abrigo_id is not None:
            update_dict = {
                "abrigoId": abrigo_id,
                "abrigoNome": abrigo_nome,
            }

            collection_ref = client_firestore.collection(collection_name)
            query = collection_ref.where("search_field_name", "==", value["search_field_name"])
            results = query.stream()
            for doc in results:
                doc_ref = client_firestore.collection(collection_name).document(doc.id)
                batch.update(doc_ref, update_dict)
                count += 1
                if count % 500 == 0:
                    batch.commit()
                    print(f"Batch com {count} operações enviado.")
                    batch = client_firestore.batch()
                    count = 0

    if count > 0:
        batch.commit()
        print(f"Batch final com {count} operações enviado.")

except Exception as e:
    print(f"Error updating Firestore documents: {e}")
    collection_name = "abrigado"

### Exclusão de Abrigos e seus abrigados

In [ ]:
# Inicialização do Firebase
abrigo_collection_name = "abrigo"
abrigado_collection_name = "abrigado"

# abrigoId que será manipulado
abrigo_id = "JtcFtoURWfjlSmcChOVo"
abrigo_field_id = "abrigoId"


# Executa a exclusão de todos os abrigados de um abrigo
print(f"Excluindo todos os abrigados do abrigo {abrigo_id}")


# Query
collection_abrigado_ref = client_firestore.collection(abrigado_collection_name)
pre_delete_query = collection_abrigado_ref.where(abrigo_field_id, "==", abrigo_id)
results = pre_delete_query.stream()

# Contar documentos antes da exclusão
pre_delete_count = sum(1 for _ in pre_delete_query.stream())
print(f"Total de documentos a serem deletados: {pre_delete_count}")

# Processar a deleção em lotes
results = pre_delete_query.stream()
deleted_count = 0
batch = client_firestore.batch()
batch_count = 0

# Cria lote de exclusão
for doc in results:
    doc_ref = collection_abrigado_ref.document(doc.id)
    batch.delete(doc_ref)
    batch_count += 1
    deleted_count += 1

    # Se o lote atingir o limite de 500, fazer commit e iniciar um novo lote
    if batch_count >= 500:
        batch.commit()
        batch = client_firestore.batch()
        batch_count = 0

# Commit do último lote se houver documentos restantes
if batch_count > 0:
    batch.commit()

print(f"Total de documentos deletados: {deleted_count}")

# Exclusão do abrigo
print(f"Excluindo abrigo {abrigo_id}")
abrigo_ref = client_firestore.collection(abrigo_collection_name).document(abrigo_id)
abrigo_ref.delete()
print(f"Abrigo {abrigo_id} excluído com sucesso.")

### Correção dos abrigados em abrigo "Em nome"
O objetivo desse bloco é corrigr as importações que tiveram problemas com o nome do abrigo quando ele estva em branço. Em um dos processos de importação os abrigos foram marcados com "Em abrigo" ao invés de "Sem abrigo". Esse processo processa uma planilha xlsx de ajustes que contém o nome do abrigo e nome do abrigado. A planilha é iterada, pesquisando pelo abrigado. Quando um abrigado é encontrado então é verificado se o nome do abrigo no qual ele está registrado é `Em abrigo`. Quando essa condição é satisfeita então ele é atualizado para o abrigo `Abrigo não informado na importação`, ou seja, todas as importações que não tiveram abrigos são corrigidas para esse novo abrigo ao invés de ficarem em um estado inválido, uma vez que o abrigo `Em abrigo ` não existe.

> Existe chance de colisão de nomes, ou seja, pessoas diferentes com o mesmo nome em função da falta de informações obrigatórias (atualmente somente o nome). A correção trabalha de forma otimista, ou seja, considera que a colisão de nomes não ocorrerá ou terá um incidência muito baixa.

> **Modelo do xlsx:**
>
> Coluna A=Abrigo, B=Abrigado

In [ ]:
try:
    datasource_file = "./pipeline/.working/ajuste_nome_sem_abrigo.xlsx"
    collection_name = "abrigado"
    wrong_name = ["Em abrigo", "em abrigo"]

    # Faz leitura do arquivo de ajuste (Coluna A=Abrigo, B=Abrigado)
    xlsx = pd.ExcelFile(datasource_file)
    data_source = pd.read_excel(xlsx, header=0, names=["abrigo_nome", "abrigado_nome"], usecols=range(2))
    data_source = data_source[data_source["abrigado_nome"].notna() & (data_source["abrigado_nome"] != "")]
    data_source["search_field_name"] = data_source.apply(lambda row: normalize_nome(row["abrigado_nome"]), axis=1)
    registers = data_source.to_dict(orient="records")

    # Inicia batch de atualização
    batch = client_firestore.batch()
    count = 0

    # Registra casos onde o abrigado foi visto como sem abrigo informado mas foi registrado em outro abrigo posterirmente
    count_abrigados_atualizados_em_outras_fontes = 0

    # Itera sobre os registros do arquivo de ajuste
    for value in registers:
        abrigo_nome, abrigo_id = get_abrigo_info(value["abrigo_nome"])

        if abrigo_id is not None:

            # Cria objeto de atualização
            update_dict = {
                "abrigoId": abrigo_id,
                "abrigoNome": abrigo_nome,
            }

            # Query para buscar os documentos a serem atualizados
            collection_ref = client_firestore.collection(collection_name)
            query = collection_ref.where("search_field_name", "==", value["search_field_name"])
            results = query.stream()

            # Itera sobre os documentos de abrigados encontrados com o filtro search_field_name
            for doc in results:

                # Somente atualiza documentos que possuem o nome do abrigo incorreto
                abrigado = doc.to_dict()
                if abrigado.get("abrigoNome") in wrong_name:
                    doc_ref = client_firestore.collection(collection_name).document(doc.id)
                    batch.update(doc_ref, update_dict)
                    count += 1

                    # Se o batch atingir 500 operações, faz commit e inicia um novo batch
                    if count % 500 == 0:
                        batch.commit()
                        print(f"Batch com {count} operações enviado.")
                        batch = client_firestore.batch()
                        count = 0
                else:
                    print(f"Abrigado {abrigado['nome']} já está em outro abrigo ({abrigado['abrigoNome']}). Ignorando atualização.")
                    count_abrigados_atualizados_em_outras_fontes += 1

    # Commit do último batch se houver operações restantes
    if count > 0:
        batch.commit()
        print(f"Batch final com {count} operações enviado.")
        print(f"Total de abrigados atualizados em outras fontes: {count_abrigados_atualizados_em_outras_fontes}")

except Exception as e:
    print(f"Error updating Firestore documents: {e}")
    collection_name = "abrigado"

## Sanitização de collections

### Sanitiza dados de abrigados


In [ ]:
collection_name = "abrigado"

def extract_cpf(nome: str) -> Optional[str]:
    if nome is None:
        return None
    
    match = re.search(r"CPF (\d{11})", nome)
    return match.group(1) if match else None


def format_date_if_necessary(date):
    if pd.isna(date):
        return datetime.now().strftime("%b %d, %Y at %I:%M:%S %p UTC-3")

    if isinstance(date, datetime):
        return date

    try:
        parsed_date = datetime.strptime(date, "%Y-%m-%d %H:%M:%S")
        return parsed_date.strftime("%b %d, %Y at %I:%M:%S %p UTC-3")
    except ValueError:
        return datetime.now().strftime("%b %d, %Y at %I:%M:%S %p UTC-3")


try:
    # Prepara stream de documentos
    abrigos_ref = client_firestore.collection(collection_name)
    docs = abrigos_ref.stream()

    # Prepara o batch para atualizações
    batch = client_firestore.batch()
    count = 0
    batch_size = 500

    for doc in docs:
        user_data = doc.to_dict()

        # Referência ao documento que será atualizado
        doc_ref = abrigos_ref.document(doc.id)

        # Normaliza nome e cria search_field
        nome = user_data.get("nome")
        search_field_name = normalize_nome(nome)
        batch.update(doc_ref, {"search_field_name": search_field_name})

        # Tenta extrair o CPF do nome
        cpf = extract_cpf(nome)
        documentos = user_data.get("documentos", [])
        if cpf not in documentos and cpf is not None:
            updated_documentos = documentos + [cpf]
            batch.update(doc_ref, {"documentos": updated_documentos})

        # Ajusta o campo id para o valor do documento caso esteja vazio
        id = user_data.get("id")
        if not id:
            batch.update(doc_ref, {"id": doc.id})

        # Ajusta create_in
        create_in = user_data.get("create_in")
        create_in = format_date_if_necessary(create_in)
        batch.update(doc_ref, {"create_in": create_in})

        # Ajusta update_in
        update_in = user_data.get("update_in")
        update_in = format_date_if_necessary(update_in)
        batch.update(doc_ref, {"update_in": update_in})

        # Remove a propriedades inseridas incorretamente durante as cargas
        batch.update(doc_ref, {"created_in": firestore.DELETE_FIELD})
        batch.update(doc_ref, {"nome_normalizado": firestore.DELETE_FIELD})
        batch.update(doc_ref, {"search_field": firestore.DELETE_FIELD})
        batch.update(doc_ref, {"documento": firestore.DELETE_FIELD})
        batch.update(doc_ref, {"cpf": firestore.DELETE_FIELD})
        batch.update(doc_ref, {"codCadUnico": firestore.DELETE_FIELD})

        # Incrementa o contador de operações no batch
        count += 1

        # Se o batch atingir o tamanho máximo, comitar e começar um novo batch
        if count >= batch_size:
            batch.commit()
            batch = client_firestore.batch()
            count = 0

    # Comitar qualquer operação restante no batch final
    if count > 0:
        batch.commit()

    print("Atualização em batch concluída.")
except Exception as e:
    print(f"Ocorreu um erro na atualização em batch: {e}")

### Sanitiza dados de Abrigos


In [ ]:
collection_name = "abrigo"


try:
    # Prepara stream de documentos
    abrigos_ref = client_firestore.collection(collection_name)
    docs = abrigos_ref.stream()

    # Prepara o batch para atualizações
    batch = client_firestore.batch()
    count = 0
    batch_size = 500

    for doc in docs:
        user_data = doc.to_dict()

        # Referência ao documento que será atualizado
        doc_ref = abrigos_ref.document(doc.id)

        # Remove NaN de vagas e seta para null se for o caso
        vagas = user_data.get("vagas")
        if vagas is not None and (isinstance(vagas, float) and math.isnan(vagas)):
            vagas = None
            batch.update(doc_ref, {"vagas": None})
            count += 1

        # Seta None quando vaga for uma string vazia
        if vagas == "":
            vagas = None
            batch.update(doc_ref, {"vagas": None})
            count += 1

        # Converte 'vagas' para inteiro se for uma string numérica
        if isinstance(vagas, str) and vagas.isdigit():
            batch.update(doc_ref, {"vagas": int(vagas)})
            count += 1

        # Remove NaN de vagas_ocupadas e seta para null se for o caso
        vagas_ocupadas = user_data.get("vagas_ocupadas")
        if vagas_ocupadas is not None and (isinstance(vagas_ocupadas, float) and math.isnan(vagas_ocupadas)):
            vagas_ocupadas = None
            batch.update(doc_ref, {"vagas_ocupadas": None})
            count += 1

        # Seta None quando vagas_ocupadas for uma string vazia
        if vagas_ocupadas == "":
            vagas_ocupadas = None
            batch.update(doc_ref, {"vagas_ocupadas": None})
            count += 1

        # Converte 'vagas_ocupadas' para inteiro se for uma string numérica
        if isinstance(vagas_ocupadas, str) and vagas_ocupadas.isdigit():
            batch.update(doc_ref, {"vagas_ocupadas": int(vagas_ocupadas)})
            count += 1

        # Ajusta telefone persisitdo como array
        telefone = user_data.get("telefone")
        if isinstance(telefone, list) and len(telefone) >= 1:
            batch.update(doc_ref, {"telefone": telefone[0]})
            count += 1
            
        # Ajusta nome persisitido como array
        nome = user_data.get("nome")
        if isinstance(nome, list) and len(nome) >= 1:
            batch.update(doc_ref, {"nome": nome[0]})
            count += 1    
            
        # Ajusta city persisitido como array
        city = user_data.get("city")
        if isinstance(city, list) and len(city) >= 1:
            batch.update(doc_ref, {"city": city[0]})
            count += 1                

        # Se o batch atingir o tamanho máximo, comitar e começar um novo batch
        if count >= batch_size:
            batch.commit()
            batch = client_firestore.batch()
            count = 0

    # Comitar qualquer operação restante no batch final
    if count > 0:
        batch.commit()

    print("Atualização em batch concluída.")
except Exception as e:
    print(f"Ocorreu um erro na atualização em batch: {e}")

### Sanitização de documentos

In [ ]:
collection_name = "abrigado"


def extract_cpf(nome: str) -> Optional[str]:
    if nome is None:
        return None

    # CPF Patterns
    cpf_patterns = "(?:CPF)?[^\d]?([\(\[\{]?(\d{3}[\.\-\s,]?){3}\d{2}[\)\]\}]?|\d{11})"

    match = re.search(cpf_patterns)
    return match.group(1) if match else None


def format_date_if_necessary(date):
    if pd.isna(date):
        return datetime.now().strftime("%b %d, %Y at %I:%M:%S %p UTC-3")

    if isinstance(date, datetime):
        return date

    try:
        parsed_date = datetime.strptime(date, "%Y-%m-%d %H:%M:%S")
        return parsed_date.strftime("%b %d, %Y at %I:%M:%S %p UTC-3")
    except ValueError:
        return datetime.now().strftime("%b %d, %Y at %I:%M:%S %p UTC-3")


try:
    # Prepara stream de documentos
    abrigos_ref = client_firestore.collection(collection_name)
    docs = abrigos_ref.stream()

    # Prepara o batch para atualizações
    batch = client_firestore.batch()
    count = 0
    batch_size = 500

    for doc in docs:
        user_data = doc.to_dict()

        # Referência ao documento que será atualizado
        doc_ref = abrigos_ref.document(doc.id)

        # Normaliza nome e cria search_field
        nome = user_data.get("nome")
        search_field_name = normalize_nome(nome)
        batch.update(doc_ref, {"search_field_name": search_field_name})

        # Tenta extrair o CPF do nome
        cpf = extract_cpf(nome)
        documentos = user_data.get("documentos", [])
        if cpf not in documentos:
            updated_documentos = documentos + [cpf]
            batch.update(doc_ref, {"documentos": updated_documentos})

        # Ajusta o campo id para o valor do documento caso esteja vazio
        id = user_data.get("id")
        if not id:
            batch.update(doc_ref, {"id": doc.id})

        # Ajusta create_in
        create_in = user_data.get("create_in")
        create_in = format_date_if_necessary(create_in)
        batch.update(doc_ref, {"create_in": create_in})

        # Ajusta update_in
        update_in = user_data.get("update_in")
        update_in = format_date_if_necessary(update_in)
        batch.update(doc_ref, {"update_in": update_in})

        # Remove a propriedades inseridas incorretamente durante as cargas
        batch.update(doc_ref, {"created_in": firestore.DELETE_FIELD})
        batch.update(doc_ref, {"nome_normalizado": firestore.DELETE_FIELD})
        batch.update(doc_ref, {"search_field": firestore.DELETE_FIELD})
        batch.update(doc_ref, {"documento": firestore.DELETE_FIELD})
        batch.update(doc_ref, {"cpf": firestore.DELETE_FIELD})
        batch.update(doc_ref, {"codCadUnico": firestore.DELETE_FIELD})

        # Incrementa o contador de operações no batch
        count += 1

        # Se o batch atingir o tamanho máximo, comitar e começar um novo batch
        if count >= batch_size:
            batch.commit()
            batch = client_firestore.batch()
            count = 0

    # Comitar qualquer operação restante no batch final
    if count > 0:
        batch.commit()

    print("Atualização em batch concluída.")
except Exception as e:
    print(f"Ocorreu um erro na atualização em batch: {e}")

### Contagem de documentos por tipos de estrutura de documento encontrada

In [ ]:
abrigo_ref = client_firestore.collection('abrigo')

# Dicionário para armazenar as estruturas encontradas e suas contagens
estruturas = {}
# Dicionário para rastrear tipos diferentes para a mesma propriedade, excluindo NoneType
tipos_por_propriedade = {}

# Itera sobre os documentos na coleção
for doc in abrigo_ref.stream():
    doc_data = doc.to_dict()
    # Cria uma representação da estrutura do documento
    estrutura = {}
    for key, value in doc_data.items():
        tipo = type(value).__name__
        if tipo != 'NoneType':  # Ignora NoneType
            estrutura[key] = tipo

            # Rastrear os tipos de cada propriedade, excluindo NoneType
            if key in tipos_por_propriedade:
                tipos_por_propriedade[key].add(tipo)
            else:
                tipos_por_propriedade[key] = {tipo}

    # Converte a estrutura em uma string para usá-la como chave no dicionário
    estrutura_str = str(sorted(estrutura.items()))

    # Atualiza a contagem de documentos com essa estrutura
    if estrutura_str in estruturas:
        estruturas[estrutura_str] += 1
    else:
        estruturas[estrutura_str] = 1

# Imprime a quantidade total de documentos por cada estrutura, e a contagem primeiro
for estrutura, count in estruturas.items():
    print(f"Contagem: {count}, Estrutura: {estrutura}")

# Imprime os diferentes tipos encontrados para cada propriedade, excluindo NoneType e propriedades com apenas um tipo
print("\nTipos diferentes encontrados por propriedade (excluindo NoneType e propriedades com um único tipo):")
for prop, tipos in tipos_por_propriedade.items():
    if len(tipos) > 1:  # Exclui propriedades com apenas um tipo
        print(f"Propriedade: {prop}, Tipos: {tipos}, Contagem de Tipos: {len(tipos)}")


## Backps e exportações

### Backup de collections

In [ ]:
# Definir o nome da coleção e o arquivo de saída
collection_name = 'users'
output_file = '../temp/collection_users.json'
output_file_csv = '../temp/collection_users.csv'

def datetime_converter(o):
    if isinstance(o, firestore.datetime.datetime):
        return o.isoformat()
    raise TypeError("Object of type 'DatetimeWithNanoseconds' is not JSON serializable")

def backup_collection_to_json(collection_name, output_file):
    try:
        collection_ref = client_firestore.collection(collection_name)
        docs = collection_ref.stream()
        
        # Criar uma lista para armazenar todos os documentos como dicionários
        all_docs = []
        for doc in docs:
            doc_dict = doc.to_dict()
            # Converter todos os DatetimeWithNanoseconds para strings ISO 8601
            for key, value in doc_dict.items():
                if isinstance(value, datetime):
                    doc_dict[key] = value.isoformat()
            all_docs.append(doc_dict)
        
        # Escrever os dados em um arquivo JSON
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(all_docs, f, default=datetime_converter, ensure_ascii=False, indent=4)
            
        # Gera csv
        df = pd.read_json(output_file)

        # Exporta o DataFrame para um arquivo CSV
        df.to_csv(output_file_csv, index=False)    
        
        print(f"Backup da coleção '{collection_name}' completo. Dados salvos em '{output_file}'.")
    except Exception as e:
        print(f"Ocorreu um erro durante o backup: {e}")

# Executar o backup
backup_collection_to_json(collection_name, output_file)


### Exportação de abrigados para orgãos de segurança

In [ ]:
# Definir o nome da coleção e o arquivo de saída
collection_name = "abrigado"
output_file = "../temp/abrigado_export.xlsx"


def datetime_converter(o):
    if isinstance(o, datetime):
        return o.__str__()


def load_collection(collection_name):
    try:
        collection_ref = client_firestore.collection(collection_name)
        docs = collection_ref.stream()

        all_docs = []
        for doc in docs:
            try:
                doc_dict = doc.to_dict()
                for key, value in doc_dict.items():
                    if isinstance(value, datetime):
                        doc_dict[key] = value.isoformat()
                all_docs.append(
                    doc_dict
                )  # Certifique-se que esta linha está dentro do loop
            except Exception as e:
                print(f"Erro ao processar documento {doc.id}: {e}")

        if not all_docs:  # Checa se a lista está vazia
            print("Nenhum documento foi carregado da coleção.")
        return all_docs

    except Exception as e:
        print(f"Erro ao carregar coleção: {e}")
        return []


def export_to_excel(data, output_file):
    try:
        # Criando um DataFrame a partir dos dados
        df = pd.DataFrame(data)

        if df.empty:  # Verifica se o DataFrame está vazio
            print("O DataFrame está vazio. Nenhum dado para exportar.")
        else:
            # Salvando o DataFrame em um arquivo Excel
            df.to_excel(output_file, index=False)
            print(f"Dados exportados para {output_file}")

    except Exception as e:
        print(f"Erro ao salvar arquivo Excel: {e}")


# Carregando dados da coleção
data = load_collection(collection_name)

# Exportando dados para o arquivo Excel
export_to_excel(data, output_file)

### Exportação de abrigos para orgãos de segurança

In [ ]:
# Definir o nome da coleção e o arquivo de saída
collection_name = "abrigo"
output_file = "../temp/abrigo_export.xlsx"


def datetime_converter(o):
    if isinstance(o, datetime):
        return o.__str__()


def load_collection(collection_name):
    try:
        collection_ref = client_firestore.collection(collection_name)
        docs = collection_ref.stream()

        all_docs = []
        for doc in docs:
            try:
                doc_dict = doc.to_dict()
                for key, value in doc_dict.items():
                    if isinstance(value, datetime):
                        doc_dict[key] = value.isoformat()
                all_docs.append(
                    doc_dict
                )  # Certifique-se que esta linha está dentro do loop
            except Exception as e:
                print(f"Erro ao processar documento {doc.id}: {e}")

        if not all_docs:  # Checa se a lista está vazia
            print("Nenhum documento foi carregado da coleção.")
        return all_docs

    except Exception as e:
        print(f"Erro ao carregar coleção: {e}")
        return []


def export_to_excel(data, output_file):
    try:
        # Criando um DataFrame a partir dos dados
        df = pd.DataFrame(data)

        if df.empty:  # Verifica se o DataFrame está vazio
            print("O DataFrame está vazio. Nenhum dado para exportar.")
        else:
            # Salvando o DataFrame em um arquivo Excel
            df.to_excel(output_file, index=False)
            print(f"Dados exportados para {output_file}")

    except Exception as e:
        print(f"Erro ao salvar arquivo Excel: {e}")


# Carregando dados da coleção
data = load_collection(collection_name)

# Exportando dados para o arquivo Excel
export_to_excel(data, output_file)

## Exercício de integração com Airtable

In [ ]:
from pyairtable import Table

api_key = sosrs_airtable_api_key
base_id = 'appfQT2vfWc6sZRAx'
table_name = 'shrH5MuXAGjBqqW0j/tblJ5UvIdVmYeTb8s'

table = Table(api_key, base_id, table_name)

# Buscar todos os registros
records = table.all()

# Exibir registros
for record in records:
    print(record)

## Exercícios de integração como Google Sheets

In [ ]:

import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd

# Define o escopo da API
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']

# Faz a autenticação usando o arquivo de credenciais
creds = ServiceAccountCredentials.from_json_keyfile_name(sosrs_firestore_path, scope)
client = gspread.authorize(creds)

# Define a planilha
spreadsheet_id = "1JrnFKHELLMVpYdsnndjIWa8zTHaG0B62ESV4JHWwc2w"

# Abre a planilha pelo nome e seleciona a aba específica
sheet = client.open(spreadsheet_id).worksheet("Respostas ao formulário 1")  # Substitua 'master_data_sheet' pela aba desejada

# Obtém todos os dados da planilha em formato de lista de listas
data = sheet.get_all_values()
print(data)

# Converte para DataFrame
data_source = pd.DataFrame(data[1:], columns=data[0])

# Se você deseja usar nomes de colunas específicos e selecionar certas colunas
data_source.columns = column_names  # Assegure-se de que 'column_names' tem o mesmo número de colunas que você está lendo
data_source = data_source.iloc[:, :number_of_columns]  # Seleciona um número específico de colunas

# Preenche valores NA com strings vazias
data_source.fillna("", inplace=True)

## Conversão de PDF em Excel

#### 1 página por aba

In [ ]:
import tabula
import pandas as pd


# Caminho do arquivo PDF
file_path = r"E:\thiagoborba.me\sosrs\data_management\data_importing\temp\abrigados_bom_conselho.pdf"

# Converter PDF para DataFrame
# O parâmetro pages especifica as páginas a serem convertidas, 'all' para todas as páginas
dfs = tabula.read_pdf(file_path, pages="all", multiple_tables=True)

combined_df = pd.concat(dfs, ignore_index=True)

# Salvar cada tabela como uma planilha no arquivo Excel
writer = pd.ExcelWriter(r"E:\thiagoborba.me\sosrs\data_management\data_importing\temp\abrigados_bom_conselho.xlsx", engine="openpyxl")

if dfs:
    for i, df in enumerate(dfs):
        df.to_excel(writer, sheet_name=f'Sheet{i+1}')
    print("A conversão foi concluída com sucesso!")
else:
    print("Nenhuma tabela encontrada no PDF.")
writer.close()

print("A conversão foi concluída com sucesso!")

### Concatena abas de uma planilha

In [ ]:
import pandas as pd

# Caminho para o arquivo Excel
file_path = r'E:\thiagoborba.me\sosrs\data_management\data_importing\temp\todas_planilhas\working\Familias SESC Protasio Alves.xlsx'

# Ler todas as abas do arquivo Excel
# Isso cria um dicionário de DataFrames
all_sheets = pd.read_excel(file_path, sheet_name=None)

# Concatenar todos os DataFrames em um único DataFrame
# Use ignore_index=True para redefinir os índices no DataFrame resultante
combined_df = pd.concat(all_sheets.values(), ignore_index=True)

# Criar um ExcelWriter para salvar o resultado
writer = pd.ExcelWriter(r'E:\thiagoborba.me\sosrs\data_management\data_importing\temp\todas_planilhas\working\Familias SESC Protasio Alves_merged.xlsx', engine='openpyxl')

# Escrever o DataFrame concatenado em uma nova aba no arquivo Excel
combined_df.to_excel(writer, sheet_name='Dados Concatenados')

# Salvar e fechar o arquivo Excel
writer.close()

print("Todas as abas foram concatenadas com sucesso!")
